### Links

https://www.wikidata.org/wiki/Wikidata:Data_access <br>
https://www.wikidata.org/wiki/Wikidata:REST_API <br>
https://www.mediawiki.org/wiki/API:Ratelimit <br>
https://doc.wikimedia.org/Wikibase/master/js/rest-api/#/ <br>
https://doc.wikimedia.org/Wikibase/master/php/repo_rest-api_README.html <br>
https://doc.wikimedia.org/Wikibase/master/php/repo_rest-api_README.html <br>

### Functions

In [1]:
import requests
import time
from rdflib import Graph, URIRef, Namespace

In [2]:
subject_ids = [
    "Q3918",   # University
    "Q9091606", # Universidad Privada del Valle
    "Q750",    # Bolivia
    "Q3028249", # UADE
    "Q414",    # Argentina
    "Q1486",   # Buenos Aires
    "Q5770918", # City of Argentina
    "Q12585"  # Latin America
]

In [3]:
predicates_ids = [
    "P31",   # Instance of
    "P17",   # Country
    "P361",  # Part of
    "P131",  # Located in the administrative territorial entity
    "P1376"  # Capital of
]

In [4]:
def get_subject(subject_id):       
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{subject_id}.json"
    response = requests.get(url)
    data = response.json()

    return data

In [5]:
def make_graph(subject_ids,predicates_ids):
    graph = []
    for subject_id in subject_ids:     
        current_subject = get_subject(subject_id)
        predicates = list(current_subject["entities"][subject_id]["claims"].keys())
        filtered_predicates = [x for x in predicates if x in predicates_ids]
        for predicate in filtered_predicates:
            try:
                current_object = current_subject["entities"][subject_id]["claims"][predicate][0]["mainsnak"]["datavalue"]["value"]["id"]
                if current_object in subject_ids:
                    graph.append([subject_id,predicate,current_object])
            except:
                pass

    return graph

### Graph Creation

In [6]:
start_time = time.time()
graph = make_graph(subject_ids,predicates_ids)
end_time = time.time()

In [7]:
execution_time_ms = (end_time - start_time) * 1000

print(f"Function execution time: {execution_time_ms:.2f} ms")

Function execution time: 5685.65 ms


In [8]:
g = Graph()

WIKIDATA = Namespace("http://www.wikidata.org/entity/")
WIKIDATA_PROP = Namespace("http://www.wikidata.org/prop/direct/")

for triple in graph:
    subject_id, predicate_id, object_id = triple
    
    subject = URIRef(WIKIDATA[subject_id])
    predicate = URIRef(WIKIDATA_PROP[predicate_id])
    obj = URIRef(WIKIDATA[object_id])
    
    g.add((subject, predicate, obj))

### Graph Plot

In [9]:
print(g.serialize(format="nt"))

<http://www.wikidata.org/entity/Q5770918> <http://www.wikidata.org/prop/direct/P17> <http://www.wikidata.org/entity/Q414> .
<http://www.wikidata.org/entity/Q9091606> <http://www.wikidata.org/prop/direct/P31> <http://www.wikidata.org/entity/Q3918> .
<http://www.wikidata.org/entity/Q1486> <http://www.wikidata.org/prop/direct/P131> <http://www.wikidata.org/entity/Q414> .
<http://www.wikidata.org/entity/Q1486> <http://www.wikidata.org/prop/direct/P1376> <http://www.wikidata.org/entity/Q414> .
<http://www.wikidata.org/entity/Q750> <http://www.wikidata.org/prop/direct/P17> <http://www.wikidata.org/entity/Q750> .
<http://www.wikidata.org/entity/Q414> <http://www.wikidata.org/prop/direct/P361> <http://www.wikidata.org/entity/Q12585> .
<http://www.wikidata.org/entity/Q414> <http://www.wikidata.org/prop/direct/P17> <http://www.wikidata.org/entity/Q414> .
<http://www.wikidata.org/entity/Q3028249> <http://www.wikidata.org/prop/direct/P17> <http://www.wikidata.org/entity/Q414> .
<http://www.wikidat